In [75]:
import os
import scipy as sp
import rsmf
import matplotlib
from matplotlib import pyplot as plt
from postProcessing.importData import *
import tensorflow as tf
import networkx as nx


NAME = "64cl_sv_LV_3" #11-vortex case 6422JO-1.70c12-1.h5 ; 7-vortex case #4822W-1, 11-vortex case 6422JO-1.70c19-1.h5 ; 4822J1, 4822J, 4822JL-1, 4822JL-10, 64MJO-1.70c19.h5
# first type - 64cl_v, 64cl_11, 64_cl_18, 64cl_LV, 64cl_LV18
# second type - 64cl_v_s3, 64cl_s_18, 64cl_s_11, 64cl_s_7, 64cl_s_3, 64cl_s_3-1, 64cl_s_3-1-1
# third type - 64cl_sv_2-2, 64cl_sv_18, 64cl_sv_11, 64cl_sv_7, 64cl_sv_LV_11, 64cl_sv_LV_5
# fourth type 64cl_sv_4
PATH = "./simulations/" + NAME
if(os.path.isdir(NAME)):
    print("Enter a valid file name")
else:
    PATH += ".h5"
    data = dataProcessing(PATH)

b'float'
SELF CONSISTENT [GAUGE, HARTREE]  ==> [True, False]
The current simulation does not export HoppingFields
Note: Magnetic free energy density not added to free energy density because defined on plaquettes


In [76]:
class ScSquareNC:
    def __init__(self, sample_size, mu,
                 init_deltas=None,
                 is_periodic=False,
                 dt=tf.dtypes.complex128):
        self.sample_size = sample_size
        self.N = int(tf.math.reduce_prod(sample_size))
        self.mu = mu
        self.dt = dt
        self.is_periodic = is_periodic
        self._gen_sample()

    def _gen_sample(self):
        g1 = nx.grid_graph([range(el) for el in list(self.sample_size)]
                           + [range(1)],
                           periodic=self.is_periodic)
        for edge in g1.edges(data=True):
            edge[2]['weight'] = 1

        i = 0
        for node in g1:
            g1.add_edge(node, node, weight=-self.mu[0, i])
            i += 1

        g2 = nx.grid_graph([range(el) for el in list(self.sample_size)]
                           + [range(1, 2)],
                           periodic=self.is_periodic)

        for edge in g2.edges(data=True):
            edge[2]['weight'] = -1

        i = 0
        for node in g2:
            g2.add_edge(node, node, weight=self.mu[1, i])
            i += 1

        latt = nx.compose(g1, g2)
        self.sample = latt
        self.hamiltonian = tf.constant(nx.to_numpy_array(self.sample),
                                                         dtype=self.dt)


    @tf.function()
    def exact_diag_coeff(self, n_phases, n_samples, norm, deltas=None):
        N = self.N
        dt = self.dt
        batch_size = tf.constant((n_samples, n_phases))
        (sample_ind,
        phase_ind,
        site_ind) = tf.meshgrid(tf.range(n_samples),
                                tf.range(n_phases),
                                tf.range(N), indexing='ij')
        mult_ham = tf.tensordot(tf.ones(batch_size, dtype=dt),
                                self.hamiltonian, axes=0)
        mult_ind_up = tf.transpose(tf.stack((sample_ind,
                                             phase_ind,
                                             site_ind,
                                             site_ind + N)),
                                   perm=(1, 2, 3, 0))
        mult_ind_down = tf.transpose(tf.stack((sample_ind,
                                               phase_ind,
                                               site_ind + N,
                                               site_ind)),
                                     perm=(1, 2, 3, 0))
        deltas_size = tf.constant((n_samples, n_phases, N))
        mult_ham = tf.tensor_scatter_nd_update(mult_ham, mult_ind_up,
                                               tf.math.conj(deltas))
        mult_ham = tf.tensor_scatter_nd_update(mult_ham, mult_ind_down,
                                               deltas)
        e, v = tf.linalg.eigh(mult_ham)
        return e, v

In [77]:
def df(E, T, Emax, V):
    return -np.exp((E + V) / T * Emax / 16) / T * Emax / 16 / (1 + np.exp((E + V) / T * Emax / 16)) ** 2

In [78]:
def tunneling_im_calc_step_1(data):
    system = ScSquareNC(data.D_1.shape, np.zeros((3, *data.D_1.shape)).reshape((3, -1)))
    deltas = np.array([[data.D_1.flatten(), data.D_2.flatten()]], dtype=np.complex128)
    Emax = data.hamiltonian.Emax
    N = len(data.D_1.flatten())
    e_full, v_full = system.exact_diag_coeff(2, 1, Emax, deltas=deltas)
    v, u = v_full[:, :, N:, N:], v_full[:, :, :N, N:]
    e_1, e_2 = e_full[:, :, N:], e_full[:, :, N:]
    return v, u, e_1, e_2, Emax

def tunneling_im_calc_step_2(u, v, e_1, e_2, T, Emax, V=0):
    dIdV0 = -tf.math.reduce_sum(tf.math.abs(u)[0, 0] ** 2 * df(tf.math.abs(e_1), T, Emax, V)[0, 0] +
                              tf.math.abs(v)[0, 0] ** 2 * df(tf.math.abs(e_2), T, Emax, -V)[0, 0], axis=-1)
    dIdV1 = -tf.math.reduce_sum(tf.math.abs(u)[0, 1] ** 2 * df(tf.math.abs(e_1), T, Emax, V)[0, 1] +
                              tf.math.abs(v)[0, 1] ** 2 * df(tf.math.abs(e_2), T, Emax, -V)[0, 1], axis=-1)

    return (dIdV0 + dIdV1 ).numpy().reshape(64, 64)

In [79]:
v, u, e_1, e_2, Emax = tunneling_im_calc_step_1(data)

In [80]:
data.cond_0 = tunneling_im_calc_step_2(v, u, e_1, e_2, data.hamiltonian.T, Emax, V=0.0)
data.cond_1 = tunneling_im_calc_step_2(v, u, e_1, e_2, data.hamiltonian.T, Emax, V=0.7)

In [81]:
fmt = rsmf.setup(r"\documentclass[aps, prl, superscriptaddress, twocolumn, letterpaper, 10pt, amsfonts, amsmath, amssymb, groupedaddress, longbibliography]{revtex4-2}")

In [87]:
fig = fmt.figure(aspect_ratio=0.73)
bl = plt.rcParams['axes.linewidth']
gs = fig.add_gridspec(2,4)
ax1= fig.add_subplot(gs[0, 0])
ax2= fig.add_subplot(gs[1, 0])
ax3= fig.add_subplot(gs[0, 1])
ax4= fig.add_subplot(gs[1, 1])
ax5= fig.add_subplot(gs[0, 2])
ax6= fig.add_subplot(gs[1, 2])
ax7= fig.add_subplot(gs[0, 3])
ax8= fig.add_subplot(gs[1, 3])

shrink = 2/3

#ax1.scatter(dist / 48, ((energies - e_v - 1.001 * energies_b) / (e_v - e_u)), label = 'total')
plt.set_cmap('Oranges')
ax1.imshow(np.abs(data.D_1))
ax1.text(0.0, 1.0, r'$\left|\Delta_1\right|$', verticalalignment='top', bbox=dict(facecolor='1', edgecolor='0', pad=3.0, linewidth=bl))
cbar = fig.colorbar(ax1.imshow(np.abs(data.D_1)), location='top', ax=ax1, drawedges=False, shrink=shrink, aspect=10, pad = 0.05)
cbar.ax.tick_params(direction='out', length=3, labelsize='small', labelrotation=60)

plt.set_cmap('twilight')
ax2.imshow(np.angle(data.D_1))
ax2.text(0.0, 1.0, r'$\varphi_1$', verticalalignment='top', bbox=dict(facecolor='1', edgecolor='0', pad=3.0, linewidth=bl))
cbar = fig.colorbar(ax2.imshow(np.angle(data.D_1)), location='bottom', ax=ax2, drawedges=False, shrink=shrink, aspect=10, pad = 0.05)
cbar.ax.tick_params(direction='out', length=3, labelsize='small', labelrotation=60)
cbar.set_ticks(ticks=[-2 * np.pi / 3, 0, 2 * np.pi / 3], labels=[r'$-2\pi/3$', r'$0$', r'$-2\pi/3$'])

plt.set_cmap('Greens')
ax3.imshow(np.abs(data.D_2))
ax3.text(0.0, 1.0, r'$\left|\Delta_2\right|$', verticalalignment='top', bbox=dict(facecolor='1', edgecolor='0', pad=3.0, linewidth=bl))
cbar = fig.colorbar(ax3.imshow(np.abs(data.D_2)), location='top', ax=ax3, drawedges=False, shrink=shrink, aspect=10, pad = 0.05)
cbar.ax.tick_params(direction='out', length=3, labelsize='small', labelrotation=60)

plt.set_cmap('twilight')
ax4.imshow(np.angle(data.D_2))
ax4.text(0.0, 1.0, r'$\varphi_2$', verticalalignment='top', bbox=dict(facecolor='1', edgecolor='0', pad=3.0, linewidth=bl))
cbar = fig.colorbar(ax4.imshow(np.angle(data.D_2)), location='bottom', ax=ax4, drawedges=False, shrink=shrink, aspect=10, pad = 0.05)
cbar.ax.tick_params(direction='out', length=3, labelsize='small', labelrotation=60)
cbar.set_ticks(ticks=[-2 * np.pi / 3, 0, 2 * np.pi / 3], labels=[r'$-2\pi/3$', r'$0$', r'$-2\pi/3$'])

plt.set_cmap('Purples')
ax5.imshow(data.B)
ax5.text(0.0, 1.0, r'$B$', verticalalignment='top', bbox=dict(facecolor='1', edgecolor='0', pad=3.0, linewidth=bl))
cbar = fig.colorbar(ax5.imshow(data.B), location='top', ax=ax5, drawedges=False, shrink=shrink, aspect=10, pad = 0.05)
cbar.ax.tick_params(direction='out', length=3, labelsize='small', labelrotation=60)

plt.set_cmap('Reds')
ax6.imshow(data.Jx ** 2 + data.Jy ** 2)
ax6.text(0.0, 1.0, r'$\left|J\right|$', verticalalignment='top', bbox=dict(facecolor='1', edgecolor='0', pad=3.0, linewidth=bl))
cbar = fig.colorbar(ax6.imshow(data.Jx ** 2 + data.Jy ** 2), location='bottom', ax=ax6, drawedges=False, shrink=shrink, aspect=10, pad = 0.05)
cbar.ax.tick_params(direction='out', length=3, labelsize='small', labelrotation=60)

plt.set_cmap('Greys')
ax7.imshow(data.cond_0)
ax7.text(0.0, 1.0, r'$\sigma\left(0\right)$', verticalalignment='top', bbox=dict(facecolor='1', edgecolor='0', pad=3.0, linewidth=bl))
cbar = fig.colorbar(ax7.imshow(data.cond_0), location='top', ax=ax7, drawedges=False, shrink=shrink, aspect=10, pad = 0.05)
cbar.ax.tick_params(direction='out', length=3, labelsize='small', labelrotation=60)

ax8.imshow(data.cond_1)
ax8.text(0.0, 1.0, r'$\sigma\left(V_1\right)$', verticalalignment='top', bbox=dict(facecolor='1', edgecolor='0', pad=3.0, linewidth=bl))
cbar = fig.colorbar(ax8.imshow(data.cond_1), location='bottom', ax=ax8, drawedges=False, shrink=shrink, aspect=10, pad = 0.05)
cbar.ax.tick_params(direction='out', length=3, labelsize='small', labelrotation=60)

ax1.xaxis.set_visible(False)
ax1.yaxis.set_visible(False)
ax2.xaxis.set_visible(False)
ax2.yaxis.set_visible(False)
ax3.xaxis.set_visible(False)
ax3.yaxis.set_visible(False)
ax4.yaxis.set_visible(False)
ax4.xaxis.set_visible(False)
ax5.xaxis.set_visible(False)
ax5.yaxis.set_visible(False)
ax6.xaxis.set_visible(False)
ax6.yaxis.set_visible(False)
ax7.xaxis.set_visible(False)
ax8.xaxis.set_visible(False)

ax7.yaxis.tick_right()
ax8.yaxis.tick_right()


fig.tight_layout() 

plt.subplots_adjust(hspace=0.2, wspace=-0.25)
plt.savefig('images/Vortex_cluster_3n.pdf')
plt.show()

/tmp/ipykernel_27956/3031296750.py:88: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()
